In [1]:
import openai
import pandas as pd
import numpy as np
from typing import List

In [1]:
from faff_qa import process_answer

In [2]:
process_answer("What is the capital of France?", "The capital of France is Paris.")

AttributeError: 'AddableValuesDict' object has no attribute 'final_answer'

In [3]:
import re

def extract_links(text):
    # Regular expression to match URLs more accurately
    url_pattern = re.compile(
        r'\b(?:https?://|www\.)\S+\b|'  # Matches http://, https://, or www.
        r'\b[a-z0-9-]+(?:\.[a-z]{2,})+\b',  # Matches domain-like patterns (e.g., example.com)
        re.IGNORECASE
    )
    
    # Find all matches in the text
    links = url_pattern.findall(text)
    return links

In [4]:
from langgraph.graph import Graph
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-20240620")

In [8]:
def gram_func1(state):
    messages = state['messages']
    user_input = messages[-1]
    complete_query = """You are an English writer assistant for a customer support system.
    
    Find all the grammatical mistakes in the provided text by the user and write the correct version with explanation for each one
            \n Here is the user text: """ + user_input
    response = model.invoke(complete_query)
    state['messages'].append(response.content) # appending AIMessage response to the AgentState
    return state

def gram_func2(state):
    messages = state['messages']
    agent_response = messages[-1]
    complete_query = """Rewrite the text and correct all the mistakes also improve the quality of the text """ + agent_response
    response = model.invoke(complete_query)
    state['messages'].append(response.content) # appending AIMessage response to the AgentState
    output = messages[-1]
    return output

def gram_func3(state):
    messages = state['messages']
    user_input = messages[0]
    first_response = messages[1]
    second_response = messages[-1]
    output =  user_input + "\n " + first_response + "\n " + second_response
    return output

In [12]:
workflow = Graph()

workflow.add_node("agent1", gram_func1)
workflow.add_node("agent2", gram_func2)
workflow.add_node("responder", gram_func3)

workflow.add_edge('agent1', 'agent2')

workflow.set_entry_point("agent1")
workflow.set_finish_point("agent2")

app = workflow.compile()

In [17]:
user_query = "Can you order the pure silver isol from Zepto to my address?"
proposed_answer = """
"Hey, the Zepto idol at Voylla is out of stock. And the alternative for it at the Voylla stores is not pure silver.
The idol is avilable at the Giva store. 
Pls let us know if you're okay with it."
"""

In [16]:
inputs = {"messages": ["Helo am fine wat are you doin. Nice too met you"]}
app.invoke(inputs)

KeyboardInterrupt: 

In [10]:
contains_link(dummy_string)

True